## 数据探索与可视化

### 实验目标
- 学习并掌握如何使用Python进行数据探索与可视化。
- 理解并应用数据探索的基本方法，包括数据清洗、格式化和描述性统计分析。
- 使用Matplotlib和Seaborn等可视化工具，绘制不同类型的图表，分析数据分布及特征之间的关系。
- 利用pandas_profiling生成数据的交互式报告，全面了解数据的分布、缺失值和异常值。

### 实验要求

1. 使用pandas读取数据集github_bot_processed_data.csv。探索pandas的显示选项，以便查看更多数据行和列。
2. 使用head()方法查看数据的前几行，并使用info()方法查看每列的数据类型，观察两者的区别。
使用describe()方法生成数据的描述性统计信息。
3. 对数据进行格式化处理，并展示不同格式（例如，日期、货币、百分比等）的效果。
4. 对public_repos、public_gists、followers、following等列进行对数变换，并查看其影响。
5. 使用Matplotlib绘制图表
    - 绘制条形图：展示label列的类别分布。
    - 绘制堆积柱状图：展示多个布尔特征（如site_admin、company等）的分布。
    - 绘制直方图：展示log_public_repos的对数变换后的数据分布。
    - 绘制散点图：展示public_repos与followers之间的关系。
    - 绘制散点矩阵：展示多个数值型特征之间的成对关系。
6. 使用Seaborn绘制图表
    - 绘制箱线图：展示不同label类别下log_followers的分布。
    - 绘制成对图：展示不同特征之间的成对关系，并根据label分类。
    - 绘制热图：展示log_public_repos、log_public_gists、log_followers和log_following等特征之间的相关性。
    - 绘制小提琴图：展示label与log_followers之间的分布差异。
7. 使用pandas_profiling.ProfileReport()生成交互式数据分析报告，分析数据的统计分布、缺失值、异常值等。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport

# 1. 读取数据和探索Pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
data = pd.read_csv("C:/Users/Chlor/Downloads/github_bot_processed_data.csv")

# 查看数据前几行和数据类型
print(data.head())
print(data.info())

# 生成数据描述性统计信息
print(data.describe())

# 2. 数据格式化处理
# 格式化日期列
for col in ['created_at', 'updated_at']:
    data[col] = pd.to_datetime(data[col])

# 添加百分比列
data['ratio'] = (data['public_gists'] / (data['public_repos'] + 1e-5)) * 100

# 3. 对数变换
log_cols = ['public_repos', 'public_gists', 'followers', 'following']
for col in log_cols:
    data[f'log_{col}'] = np.log1p(data[col])

# 查看对数变换效果
print(data[[col for col in data.columns if col.startswith('log_')]].head())

# 4. Matplotlib图表绘制
# 条形图：展示label列的类别分布
label_counts = data['label'].value_counts()
label_counts.plot(kind='bar', color='skyblue')
plt.title('Label Distribution')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# 堆积柱状图
bool_cols = ['site_admin', 'hireable']
bool_counts = data[bool_cols].apply(pd.Series.value_counts).T
bool_counts.plot(kind='bar', stacked=True, color=['blue', 'orange'], figsize=(8, 6))
plt.title('Boolean Features Distribution')
plt.xlabel('Features')
plt.ylabel('Count')
plt.show()

# 直方图：log_public_repos数据分布
plt.hist(data['log_public_repos'], bins=30, color='green', alpha=0.7)
plt.title('Log Public Repos Distribution')
plt.xlabel('Log(Public Repos)')
plt.ylabel('Frequency')
plt.show()

# 散点图：public_repos与followers关系
plt.scatter(data['public_repos'], data['followers'], alpha=0.6)
plt.title('Public Repos vs Followers')
plt.xlabel('Public Repos')
plt.ylabel('Followers')
plt.xscale('log')
plt.yscale('log')
plt.show()

# 散点矩阵
sns.pairplot(data[log_cols])
plt.show()

# 5. Seaborn图表绘制
# 箱线图：不同label类别下log_followers分布
sns.boxplot(x='label', y='log_followers', data=data)
plt.title('Log Followers by Label')
plt.show()

# 成对图：不同特征之间的关系按label分类
sns.pairplot(data, vars=log_cols, hue='label', palette='tab10')
plt.show()

# 热图：相关性
correlation_matrix = data[[f'log_{col}' for col in log_cols]].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')
plt.show()

# 小提琴图：label与log_followers分布差异
sns.violinplot(x='label', y='log_followers', data=data, palette='muted')
plt.title('Log Followers Distribution by Label')
plt.show()

# 6. Pandas Profiling生成交互式分析报告
profile = ProfileReport(data, title="GitHub Bot Data Analysis", explorative=True)
profile.to_file("github_bot_data_analysis_report.html")